In [3]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# os
import os




ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Step 1: Load Images and Labels
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

folder = "train_data"
images = load_images(folder)
labels = pd.read_csv("train_data/train_data_labels.csv", header=None).values.ravel()



In [ ]:
# Step 2: Preprocess Images
def preprocess_images(images):
    processed_images = [cv2.resize(img, (200, 200)).flatten() for img in images]
    return np.array(processed_images) / 255.0  # Normalize to [0, 1]

processed_images = preprocess_images(images)



In [ ]:
# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(processed_images, labels, test_size=0.2, random_state=42)



In [ ]:
# Step 4: Train KNN Classifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)



In [ ]:
# Step 5: Evaluate KNN Classifier
y_pred = knn.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}')
print(f'Classification Report:\n{classification_report(y_test, y_pred)}')

# P approach

In [ ]:
from torchvision import datasets, transforms

def get_mean_std(image_dir):
    pixel_values = []

    # Iterate over the image files in the directory
    for filename in os.listdir(image_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            img_path = os.path.join(image_dir, filename)
            image = Image.open(img_path)
            image = np.array(image)  # Convert the image to a NumPy array
            pixel_values.append(image.flatten())

    # Calculate mean and standard deviation
    pixel_values = np.concatenate(pixel_values, axis=0)
    mean = np.mean(pixel_values, axis=0)
    std = np.std(pixel_values, axis=0)

    print(image_dir)
    print("Mean:", mean)
    print("Standard Deviation:", std)

image_dir = "train_data"
get_mean_std(image_dir)

train_data
Mean: 103.6791615
Standard Deviation: 69.55526717982558


In [ ]:
import torch
import torch.optim as optim
from torchvision import datasets, models, transforms
import os
import csv
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Load the pre-trained model (e.g., ResNet)

model_up = models.resnet18(pretrained=True)

# Freeze all the layers in the model
for param in model_up.parameters():
    param.requires_grad = False

# Replace the final layer of the model to match the number of classes in the new dataset
num_ftrs = model_up.fc.in_features
model_up.fc = torch.nn.Linear(num_ftrs, 6)  # Adapted to 5 classes

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model_up.fc.parameters(), lr=0.001, momentum=0.9)

class CustomDataset(Dataset):
    def __init__(self, image_dir, labels_csv, transform=None):
        self.image_dir = image_dir
        self.image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))])
        
        # Using csv.reader to read labels from csv file
        with open(labels_csv, mode='r') as csv_file:
            csv_reader = csv.reader(csv_file)
            self.labels = [int(row[0]) for row in csv_reader]
        
        if len(self.image_files) != len(self.labels):
            raise ValueError("Mismatch: Number of image files does not match the number of labels.")

        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_name)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

mean = [103.6791615 / 255.0, 103.6791615 / 255.0, 103.6791615 / 255.0]  # Divide by 255 to scale to [0, 1]
std = [69.55526717982558 / 255.0, 69.55526717982558 / 255.0, 69.55526717982558 / 255.0]  # Divide by 255 to scale to [0, 1]
normalize_transform = transforms.Normalize(mean=mean, std=std)

# Define transformations for the images
train_transforms_up = transforms.Compose([
    #transforms.Resize((100, 100)),  # adjust size if necessary
    transforms.ToTensor(),
    normalize_transform
])

train_data = CustomDataset(image_dir="train_data", labels_csv="train_data/train_data_labels.csv", transform=train_transforms_up)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

# Training the model
num_epochs = 20
for epoch in range(num_epochs):  
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_up(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, running_loss/len(train_loader)))


c:\Users\leo_b\anaconda3\envs\pmba_new\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\leo_b\anaconda3\envs\pmba_new\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/20], Loss: 1.8161
Epoch [2/20], Loss: 1.5750
Epoch [3/20], Loss: 1.4462
Epoch [4/20], Loss: 1.3419
Epoch [5/20], Loss: 1.2482
Epoch [6/20], Loss: 1.2142
Epoch [7/20], Loss: 1.1452
Epoch [8/20], Loss: 1.0982
Epoch [9/20], Loss: 1.0900
Epoch [10/20], Loss: 1.0388
Epoch [11/20], Loss: 1.0288
Epoch [12/20], Loss: 0.9487
Epoch [13/20], Loss: 0.9607
Epoch [14/20], Loss: 0.9070
Epoch [15/20], Loss: 0.8738
Epoch [16/20], Loss: 0.8892
Epoch [17/20], Loss: 0.8718
Epoch [18/20], Loss: 0.8508
Epoch [19/20], Loss: 0.8475
Epoch [20/20], Loss: 0.8162


In [ ]:


# Evaluate the model after training to calculate accuracy and confusion matrix
model_up.eval()  # set the model to evaluation mode

import numpy as np



print(f'Root Mean Square Error (RMSE): {rmse:.2f}')

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model_up(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.numpy().tolist())
        all_labels.extend(labels.numpy().tolist())

# Calculate accuracy
# Calculate RMSE
errors = [(pred - actual)**2 for pred, actual in zip(all_preds, all_labels)]
rmse = np.sqrt(np.mean(errors))

# Compute confusion matrix without sklearn
num_classes = 6  # Change this to 6
confusion = [[0] * num_classes for _ in range(num_classes)]
for actual, pred in zip(all_labels, all_preds):
    confusion[actual][pred] += 1

for row in confusion:
    print(row)

# Testing on the test_data folder and outputting the classifications into a txt file

# Modify the CustomDataset for image-only mode (no labels)
class ImageOnlyDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))])
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image, img_name  # return image name too for reference

test_data = ImageOnlyDataset(image_dir="test_data", transform=train_transforms_up)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=False)

# Output classifications to a txt file
with open("classifications.csv", "w") as f:
    with torch.no_grad():
        for inputs, image_names in test_loader:
            outputs = model_up(inputs)
            _, predicted = torch.max(outputs, 1)
            for img, pred in zip(image_names, predicted.numpy()):
                f.write(f'{pred}\n')


Root Mean Square Error (RMSE): 0.42
[73, 0, 0, 0, 0, 0]
[3, 73, 3, 3, 0, 0]
[0, 4, 78, 15, 0, 0]
[0, 2, 2, 84, 0, 0]
[0, 0, 2, 29, 41, 4]
[0, 0, 2, 13, 2, 67]


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)  # Modified for regression
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [8]:
import os
import csv
import numpy as np
import torch
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import mobilenet_v2
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import random

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        if self.transform:
            image = self.transform(image)

        return image, label

base_path = ''
train_data_folder = os.path.join(base_path, 'train_data')
train_label_path = os.path.join(base_path, 'train_data_labels.csv')
test_data_folder = os.path.join(base_path, 'test_data')

with open(train_label_path, 'r') as f:
    reader = csv.reader(f)
    labels = [int(row[0]) for row in reader]

def combine_images(img_path_1, img_path_2):
    img1 = np.array(Image.open(img_path_1).convert('RGB'))
    img2 = np.array(Image.open(img_path_2).convert('RGB'))
    combined_img = np.clip(img1 + img2, 0, 255)
    return Image.fromarray(combined_img.astype('uint8'), 'RGB')

train_image_filenames = sorted(os.listdir(train_data_folder))
train_image_paths = [(os.path.join(train_data_folder, train_image_filenames[i]), os.path.join(train_data_folder, train_image_filenames[i+1])) for i in range(0, len(train_image_filenames)-1, 2) if i+1 < len(train_image_filenames)]
train_images = [combine_images(img_pair[0], img_pair[1]) for img_pair in train_image_paths]

# Train-validation split
split_ratio = 0.8
split_idx = int(len(train_images) * split_ratio)
combined_list = list(zip(train_images, labels))
random.shuffle(combined_list)
train_images, labels = zip(*combined_list)
train_images, val_images = train_images[:split_idx], train_images[split_idx:]
train_labels, val_labels = labels[:split_idx], labels[split_idx:]

class CustomDataset(Dataset):
    def _init_(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def _len_(self):
        return len(self.images)

    def _getitem_(self, idx):
        image = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CustomDataset(train_images, train_labels, transform=transform)
val_dataset = CustomDataset(val_images, val_labels, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=16)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=16)

model = mobilenet_v2(pretrained=True)
model.classifier[1] = torch.nn.Linear(model.last_channel, 1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = torch.nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")

print("Finished Training")

model.eval()
all_predictions = []
all_labels = []

for images, labels in val_loader:
    images, labels = images.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(images).squeeze(1)
    all_predictions.extend(outputs.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

rmse = np.sqrt(((np.array(all_labels) - np.array(all_predictions)) ** 2).mean())
print(f"Validation RMSE: {rmse:.4f}")

model_save_path = os.path.join(base_path, 'mobilenet_v2_trained_model.pth')
torch.save(model.state_dict(), model_save_path)

test_image_filenames = sorted(os.listdir(test_data_folder))
test_image_paths = [(os.path.join(test_data_folder, test_image_filenames[i]), os.path.join(test_data_folder, test_image_filenames[i+1])) for i in range(0, len(test_image_filenames), 2)]
test_images = [combine_images(img_pair[0], img_pair[1]) for img_pair in test_image_paths]
test_dataset = [transform(image) for image in test_images]
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=16)

model.eval()
predictions_list = []
with torch.no_grad():
    for image in test_loader:
        image = image.to(device)
        outputs = model(image).squeeze(1)
        predictions_list.extend(np.round(outputs.cpu().numpy()))

csv_output_path = os.path.join(base_path, "test_predictions.csv")
with open(csv_output_path, "w", newline="") as f:
    writer = csv.writer(f)
    for pred in predictions_list:
        writer.writerow([int(pred)])

print(f"Predictions saved in {csv_output_path}")

TypeError: object.__new__() takes exactly one argument (the type to instantiate)